In [ ]:
# !rm -rf ./wide_resnet_poly_4/val

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
%run ./datagen.py

datagen, (x_train, y_train), (x_test, y_test) = data_preparation()

In [3]:
path_model = './wide_resnet_poly_10/'
test = True
k = 10

In [ ]:
# def act(x, i):
    
# #     y = tf.math.polyval([0.12775111, 0.48709023, 0.30508906], x)

#     if i == 1:
#         y = tf.math.polyval([0.32151661, 0.49775602, 0.12350393], x)
# #         y = tf.nn.relu(x)
#     elif i == 2:
#         y = tf.math.polyval([0.15060764, 0.49911386, 0.26389601], x)
# #         y = tf.nn.relu(x)
#     elif i == 3:
#         y = tf.math.polyval([0.12234829, 0.49132487, 0.32127634], x)
# #         y = tf.nn.relu(x)
#     elif i == 4:
#         y = tf.math.polyval([0.14833134, 0.4864861,  0.26231578], x)
# #         y = tf.nn.relu(x)
#     elif i == 5:
#         y = tf.math.polyval([0.13113742, 0.49954892, 0.30331744], x)
# #         y = tf.nn.relu(x)
#     else:
#         y = tf.math.polyval([0.06568586, 0.49967003, 0.60524462], x)
# #         y = tf.nn.relu(x)

#     return y

In [ ]:
# def act(x, i):
    
#     # ax^2 + bx + c
        
#     if i == 1:
#         first_coef = tf.Variable(0.31931543, trainable=True, name='first_coef', dtype=tf.float32)
#         second_coef = tf.Variable(0.49708712, trainable=True, name='second_coef', dtype=tf.float32)
#         third_coef = tf.Variable(0.12422738, trainable=True, name='third_coef', dtype=tf.float32)
#         y = tf.math.polyval([first_coef, second_coef, third_coef], x)
        
#     elif i == 2:
#         first_coef = tf.Variable(0.14951853, trainable=True, name='first_coef', dtype=tf.float32)
#         second_coef = tf.Variable(0.49850001, trainable=True, name='second_coef', dtype=tf.float32)
#         third_coef = tf.Variable(0.26582837, trainable=True, name='third_coef', dtype=tf.float32)
#         y = tf.math.polyval([first_coef, second_coef, third_coef], x)
        
#     elif i == 3:
#         first_coef = tf.Variable(0.16431344, trainable=True, name='first_coef', dtype=tf.float32)
#         second_coef = tf.Variable(0.48894203, trainable=True, name='second_coef', dtype=tf.float32)
#         third_coef = tf.Variable(0.23802768, trainable=True, name='third_coef', dtype=tf.float32)
#         y = tf.math.polyval([first_coef, second_coef, third_coef], x)

#     elif i == 4:
#         first_coef = tf.Variable(0.14941755, trainable=True, name='first_coef', dtype=tf.float32)
#         second_coef = tf.Variable(0.48750084, trainable=True, name='second_coef', dtype=tf.float32)
#         third_coef = tf.Variable(0.26098495, trainable=True, name='third_coef', dtype=tf.float32)
#         y = tf.math.polyval([first_coef, second_coef, third_coef], x)

#     elif i == 5:
#         first_coef = tf.Variable(0.10792727, trainable=True, name='first_coef', dtype=tf.float32)
#         second_coef = tf.Variable(0.48113492, trainable=True, name='second_coef', dtype=tf.float32)
#         third_coef = tf.Variable(0.35588625, trainable=True, name='third_coef', dtype=tf.float32)
#         y = tf.math.polyval([first_coef, second_coef, third_coef], x)

#     else:
#         first_coef = tf.Variable(0.07860769, trainable=True, name='first_coef', dtype=tf.float32)
#         second_coef = tf.Variable(0.49907132, trainable=True, name='second_coef', dtype=tf.float32)
#         third_coef = tf.Variable(0.50563614, trainable=True, name='third_coef', dtype=tf.float32)
#         y = tf.math.polyval([first_coef, second_coef, third_coef], x)

#     return y

In [ ]:
# # resnet layer
# def res_layer(inputs, filter_num, filter_size, stride, is_train, activation_num, 
#               batch_norm=True, activation=True):

#     x = inputs
        

#     if batch_norm:
#         x = tf.layers.batch_normalization(x, training=is_train)

#     if activation:          
#         x = act(x, activation_num)

#     x = tf.layers.conv2d(inputs=x, filters=filter_num,  
#                          kernel_size=filter_size, strides=stride, padding='same')
        
#     return x

In [7]:
# resnet layer
def res_layer(inputs, filter_num, filter_size, stride, is_train,
              batch_norm=True, activation=True):

    x = inputs

    if batch_norm:
        x = tf.layers.batch_normalization(x, training=is_train)
#         tf.summary.histogram('Before_Act', x)
#         tf.summary.scalar('std', tf.math.reduce_std(x))
#         tf.summary.scalar('mean', tf.math.reduce_mean(x))
#         tf.summary.scalar('size', tf.size(x))
    if activation:
        x = tf.nn.relu(x)
    x = tf.layers.conv2d(inputs=x, filters=filter_num, 
                         kernel_size=filter_size, strides=stride, padding='same')
        
    return x

In [8]:
def wide_resnet(inputs, k, is_train):

    with tf.variable_scope("1st_Conv"):
        x = tf.layers.conv2d(inputs=inputs, filters=16, 
                             kernel_size=3, strides=1, padding='same')
        x = tf.layers.batch_normalization(x, training=is_train)
#         tf.summary.histogram('After_BN', x)
        x = tf.nn.relu(x)        

    # Res Blocks
    a = [1, 1, 1]
    act_num = 0

    for stack in range(len(a)):
        for block in range(a[stack]):

            with tf.variable_scope('ResBlock_%d_%d' % (stack+1, block+1)):

                batch_norm = True
                activation = True
                stride = 1
                filter_num = 16*k*(2**stack)
                if stack == 0:
                    if block == 0:
                        batch_norm = False
                        activation = False
                else:  
                    if block == 0:
                        stride = 2

                shortcut = x
                with tf.variable_scope('conv1'):
                    x = res_layer(x, filter_num, 3, stride, is_train,
                                  batch_norm=batch_norm, activation=activation)
                    act_num += 1
                    
                x = tf.layers.dropout(x, 0.1)
                
                with tf.variable_scope('conv2'):
                    x = res_layer(x, filter_num, 3, 1, is_train, act_num)
                    act_num += 1
                
                with tf.variable_scope('x_plus_shortcut'):
                    if block == 0:
                        shortcut = tf.layers.conv2d(inputs=shortcut, filters=filter_num, 
                                                    kernel_size=1, strides=stride, padding='same')
                    x = x + shortcut
    
    with tf.variable_scope("AfterResBlock"):
        x = tf.layers.batch_normalization(x, training=is_train)
#         tf.summary.histogram('Before_Act', x)
#         tf.summary.scalar('std', tf.math.reduce_std(x))
#         tf.summary.scalar('mean', tf.math.reduce_mean(x))
#         tf.summary.scalar('size', tf.size(x))
#         x = act(x, act_num)
        x = tf.nn.relu(x)
#         tf.summary.histogram('After_Act', x)
        x = tf.layers.average_pooling2d(x, pool_size=8, strides=8, padding='SAME', name='ave_pool')
        
    with tf.variable_scope("Flatten"):
        x = tf.transpose(x, perm=[0, 3, 1, 2])
        x = tf.layers.flatten(x)
    
    # crrent x.shape = (?, 256)
    with tf.variable_scope("Prediction"):
        pred = tf.layers.dense(x, units=10)
#         tf.summary.histogram('prediction', pred)
        
    return pred

In [ ]:
# def wide_resnet(inputs, k, is_train):
    
#     act_num = 0

#     with tf.variable_scope("1st_Conv"):
#         x = tf.layers.conv2d(inputs=inputs, filters=16,  
#                              kernel_size=3, strides=1, padding='same')
#         x = tf.layers.batch_normalization(x, training=is_train)
#         x = tf.nn.relu(x)
    
#     x_temp_0 = x
    
#     with tf.variable_scope('ResBlock_%d_%d' % (1, 1)):
        
#         with tf.variable_scope('conv1'):
#             x = res_layer(x, 16*k, 3, 1, is_train, act_num, 
#                           batch_norm=False, activation=False)
#             act_num += 1
            
#         x = tf.layers.dropout(x, 0.1)
        
#         with tf.variable_scope('conv2'):
#             x = res_layer(x, 16*k, 3, 1, is_train, act_num)
#             act_num += 1
    
#         with tf.variable_scope('x_plus_shortcut'):
#             shortcut = tf.layers.conv2d(inputs=x_temp_0, filters=16*k, 
#                                         kernel_size=1, strides=1, padding='same')
#             x = x + shortcut             
        
#     x_temp_1 = x
    
#     with tf.variable_scope('ResBlock_%d_%d' % (2, 1)):
        
#         with tf.variable_scope('conv1'):
#             x = res_layer(x, 32*k, 3, 2, is_train, act_num)
#         act_num += 1
        
#         x = tf.layers.dropout(x, 0.1)
                
#         with tf.variable_scope('conv2'):
#             x = res_layer(x, 32*k, 3, 1, is_train, act_num)
#         act_num += 1
        
#         with tf.variable_scope('x_plus_shortcut'):
#             shortcut = tf.layers.conv2d(inputs=x_temp_0, filters=32*k, 
#                                         kernel_size=1, strides=2, padding='same')
#             x = x + shortcut
            
#             shortcut = tf.layers.conv2d(inputs=x_temp_1, filters=32*k, 
#                                         kernel_size=1, strides=2, padding='same')
#             x = x + shortcut
        
#     x_temp_2 = x    
    
#     with tf.variable_scope('ResBlock_%d_%d' % (3, 1)):
        
#         with tf.variable_scope('conv1'):
#             x = res_layer(x, 64*k, 3, 2, is_train, act_num)
#             act_num += 1
            
#         x = tf.layers.dropout(x, 0.1)
                
#         with tf.variable_scope('conv2'):
#             x = res_layer(x, 64*k, 3, 1, is_train, act_num)
#             act_num += 1
            
#         with tf.variable_scope('x_plus_shortcut'):
            
#             shortcut = tf.layers.conv2d(inputs=x_temp_0, filters=64*k, 
#                                         kernel_size=1, strides=4, padding='same')
#             x = x + shortcut

#             shortcut = tf.layers.conv2d(inputs=x_temp_1, filters=64*k, 
#                                         kernel_size=1, strides=4, padding='same')
#             x = x + shortcut

#             shortcut = tf.layers.conv2d(inputs=x_temp_2, filters=64*k, 
#                                         kernel_size=1, strides=2, padding='same')
#             x = x + shortcut

#     with tf.variable_scope("AfterResBlock"):
#         x = tf.layers.batch_normalization(x, training=is_train)
#         x = act(x, act_num)
#         x = tf.layers.average_pooling2d(x, pool_size=8, strides=8, padding='SAME', name='ave_pool')

#     with tf.variable_scope("Flatten"):
#         x = tf.transpose(x, perm=[0, 3, 1, 2])
#         x = tf.layers.flatten(x)

#     with tf.variable_scope("Prediction"):
#         pred = tf.layers.dense(x, units=10)
        
#     return pred

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="9"

tf.reset_default_graph()

with tf.device("/gpu:9"):
    inputs = tf.placeholder(tf.float32, [None, 32, 32, 3], name='input')
    outputs = tf.placeholder(tf.float32, [None, 10], name='output')
    is_train = tf.placeholder(tf.bool, name='is_train')

    pred = wide_resnet(inputs, k, is_train)
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(outputs, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#     merged = tf.summary.merge_all()

    saver = tf.train.Saver(tf.global_variables())

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [12]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True

start_num = 0
if test:
    start_num = 5000

with tf.Session(config=config) as sess: 
    print('*****************Test stage*****************')
    saver.restore(sess, path_model+'cifar10.ckpt-211500')
#     test_writer = tf.summary.FileWriter(path_model+'test')

    testaccs = []
    for i in range(10):
        test_acc, prediction = sess.run([accuracy, pred], 
                                        {inputs: x_test[start_num+500*i:start_num+500*(i+1)], 
                                         outputs: y_test[start_num+500*i:start_num+500*(i+1)], 
                                         is_train: False})
#         test_writer.add_summary(summary, i)

        testaccs.append(test_acc)

    print('Test accuracy: {:.4f}'.format(np.mean(testaccs)))

*****************Test stage*****************
INFO:tensorflow:Restoring parameters from ./wide_resnet_poly_10/cifar10.ckpt-211500
Test accuracy: 0.9246


In [15]:
prediction[0]

array([-29.815733  ,  -0.26563138, -22.744297  , -10.487765  ,
       -27.769377  , -19.388401  , -29.83771   , -22.300894  ,
       -18.476685  ,  28.550013  ], dtype=float32)